## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

with open('D:/inputs/24/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)


with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)
LEN_TRAIN = train.shape[0]

In [ ]:
train.shape

In [ ]:
163.022018 - 161.256897 

In [ ]:
display(
    train.shape,
    train_y.shape,
    test.shape
)

In [ ]:
# 조건에 맞는 사진 개수 세는 알고리즘.
# from tqdm import tqdm
# cnt = 0
# for i in tqdm(train_y):
#     for ii in i:
#         if (ii == 0).sum() != 40:
#             cnt += 1
# cnt

## 9번 channel 가공 - 카테고리형 피쳐

In [ ]:
# train = np.delete(train, 9, axis=3)
# test = np.delete(test, 9, axis=3)

In [ ]:
with open('D:/inputs/24/rot_train.pickle', 'rb') as f:
    rot_train = pickle.load(f)
with open('D:/inputs/24/rot_train_y.pickle', 'rb') as f:
    rot_train_y = pickle.load(f)
    rot_train_y = rot_train_y.reshape(rot_train_y.shape[0], 40, 40, 1)
LEN_TRAIN = train.shape[0] + rot_train.shape[0]

train = np.vstack([train, rot_train])
train_y = np.vstack([train_y, rot_train_y])

del rot_train
del rot_train_y
gc.collect()

## all_np로 결합 후 scaling

In [ ]:
all_np = np.vstack([train, test])
del train, test

rotation_45_1_1 = all_np[:, :, :, -4] + all_np[:, :, :, -3]
rotation_45_1_2 = all_np[:, :, :, -4] - all_np[:, :, :, -3]
rotation_45_2_1 = all_np[:, :, :, -2] + all_np[:, :, :, -1]
rotation_45_2_2 = all_np[:, :, :, -2] - all_np[:, :, :, -1]

rotation_45_1_1 = rotation_45_1_1.reshape(all_np.shape[0], 40, 40,1)
rotation_45_1_2 = rotation_45_1_2.reshape(all_np.shape[0], 40, 40,1)
rotation_45_2_1 = rotation_45_2_1.reshape(all_np.shape[0], 40, 40,1)
rotation_45_2_2 = rotation_45_2_2.reshape(all_np.shape[0], 40, 40,1)

all_np = np.concatenate((all_np, rotation_45_1_1), axis=-1)
all_np = np.concatenate((all_np, rotation_45_1_2), axis=-1)
all_np = np.concatenate((all_np, rotation_45_2_1), axis=-1)
all_np = np.concatenate((all_np, rotation_45_2_2), axis=-1)

del rotation_45_1_1
del rotation_45_1_2
del rotation_45_2_1
del rotation_45_2_2


rotation_30_1_1 = all_np[:, :, :, -4] * np.cos(np.pi / 6) + all_np[:, :, :, -3] * np.sin(np.pi / 6)
rotation_30_1_2 = all_np[:, :, :, -4] * np.cos(np.pi / 6) - all_np[:, :, :, -3] * np.sin(np.pi / 6)
rotation_30_2_1 = all_np[:, :, :, -2] * np.cos(np.pi / 6) + all_np[:, :, :, -1] * np.sin(np.pi / 6)
rotation_30_2_2 = all_np[:, :, :, -2] * np.cos(np.pi / 6) - all_np[:, :, :, -1] * np.sin(np.pi / 6)

rotation_30_1_1 = rotation_30_1_1.reshape(all_np.shape[0], 40, 40,1)
rotation_30_1_2 = rotation_30_1_2.reshape(all_np.shape[0], 40, 40,1)
rotation_30_2_1 = rotation_30_2_1.reshape(all_np.shape[0], 40, 40,1)
rotation_30_2_2 = rotation_30_2_2.reshape(all_np.shape[0], 40, 40,1)

all_np = np.concatenate((all_np, rotation_30_1_1), axis=-1)
all_np = np.concatenate((all_np, rotation_30_1_2), axis=-1)
all_np = np.concatenate((all_np, rotation_30_2_1), axis=-1)
all_np = np.concatenate((all_np, rotation_30_2_2), axis=-1)

del rotation_30_1_1
del rotation_30_1_2
del rotation_30_2_1
del rotation_30_2_2


rotation_60_1_1 = all_np[:, :, :, -4] * np.cos(np.pi / 3) + all_np[:, :, :, -3] * np.sin(np.pi / 3)
rotation_60_1_2 = all_np[:, :, :, -4] * np.cos(np.pi / 3) - all_np[:, :, :, -3] * np.sin(np.pi / 3)
rotation_60_2_1 = all_np[:, :, :, -2] * np.cos(np.pi / 3) + all_np[:, :, :, -1] * np.sin(np.pi / 3)
rotation_60_2_2 = all_np[:, :, :, -2] * np.cos(np.pi / 3) - all_np[:, :, :, -1] * np.sin(np.pi / 3)

rotation_60_1_1 = rotation_60_1_1.reshape(all_np.shape[0], 40, 40,1)
rotation_60_1_2 = rotation_60_1_2.reshape(all_np.shape[0], 40, 40,1)
rotation_60_2_1 = rotation_60_2_1.reshape(all_np.shape[0], 40, 40,1)
rotation_60_2_2 = rotation_60_2_2.reshape(all_np.shape[0], 40, 40,1)

all_np = np.concatenate((all_np, rotation_60_1_1), axis=-1)
all_np = np.concatenate((all_np, rotation_60_1_2), axis=-1)
all_np = np.concatenate((all_np, rotation_60_2_1), axis=-1)
all_np = np.concatenate((all_np, rotation_60_2_2), axis=-1)

del rotation_60_1_1
del rotation_60_1_2
del rotation_60_2_1
del rotation_60_2_2


train = all_np[:LEN_TRAIN]
test = all_np[LEN_TRAIN:]
del all_np
gc.collect()


TR_SHAPE_0 = train.shape[0]
TR_SHAPE_1 = train.shape[1]
TR_SHAPE_2 = train.shape[2]
TR_SHAPE_3 = train.shape[3]


TE_SHAPE_0 = test.shape[0]
TE_SHAPE_1 = test.shape[1]
TE_SHAPE_2 = test.shape[2]
TE_SHAPE_3 = test.shape[3]

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

train = train.reshape(TR_SHAPE_0 * TR_SHAPE_1 * TR_SHAPE_2, TR_SHAPE_3)
test = test.reshape(TE_SHAPE_0 * TE_SHAPE_1 * TE_SHAPE_2, TE_SHAPE_3)
all_np = np.vstack([train, test])

del train
del test
gc.collect()

# ss = StandardScaler()
# rs = RobustScaler()
# mm = MinMaxScaler()
# all_np = ss.fit_transform(all_np)

for i in range(all_np.shape[1]):
    ss = StandardScaler()
    all_np[:,i]  = ss.fit_transform(all_np[:,i].reshape(-1, 1)).reshape(len(all_np))


train = all_np[:TR_SHAPE_0 * TR_SHAPE_1 * TR_SHAPE_2]
test = all_np[TR_SHAPE_0 * TR_SHAPE_1 * TR_SHAPE_2:]

del all_np
del ss

In [ ]:
train = train.reshape(TR_SHAPE_0 , TR_SHAPE_1 , TR_SHAPE_2, TR_SHAPE_3)
test = test.reshape(TE_SHAPE_0 , TE_SHAPE_1 , TE_SHAPE_2, TE_SHAPE_3)

In [ ]:
gc.collect()

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model

def build_model(input_layer, start_neurons):
    
    # 40 x 40 -> 20 x 20
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    return output_layer

input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model(input_layer, 32)
model = Model(input_layer, output_layer)

# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
model.compile(loss="mae", optimizer="adam", metrics=["mae"])

scores = []



In [ ]:
def loss(predicted_y, desired_y):
    predictions = tf.argmax(desired_y, axis=-1)
    pr, pr_op = tf.keras.metrics.Precision(predicted_y, predictions)
    re, re_op = tf.keras.metrics.Recall(predicted_y, predictions)
    f1 = (2 * pr * re) / (pr + re)
    return tf.reduce_mean(tf.abs(predicted_y - desired_y)) / f1

In [ ]:
model = tf.keras.models.load_model('epochs_8.h5')
model.summary()

In [ ]:
EPOCHS = 30
for _ in range(5):
    for i in range(EPOCHS):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 200)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

In [ ]:
EPOCHS = 30
for _ in range(1):
    for i in range(EPOCHS):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 400)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

In [ ]:
EPOCHS = 30
for _ in range(1):
    for i in range(1):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 400)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

In [ ]:
model.save('epochs_8.h5')

In [ ]:
print(len(scores) * 30)

In [ ]:
pred = model.predict(test)
submission = pd.read_csv('inputs/sample_submission.csv')
submission.iloc[:,1:] = pred.reshape(-1, 1600)
submission.to_csv('Dacon_baseline.csv', index = False)

**해야할 일**

1. 스케일링(Scaling) - Robust, log, Standard Scaler  
2. ~~위도 경도 피쳐를 가지고 위도+경도, 위도-경도 피쳐 만들기(45도 회전변환)~~: 30, 60도 변환 추가
3. 현재 custom metric이 작동하지 않으므로, 이 또한 수정해야함.
4. tree모델 만들고, 앙상블
5. ~~target 값에 log 취하기~~: 점수 안오름
6. 순서가 바뀌긴 했으나, EDA 작업하자.
7. 강수량 타일이 10개 미만인 것의 평균, 20개미만의 평균.... 50개 미만의 평균... 알아보자
8. ~~augumentation 사용해서 강수량이 일정수준 이상인 사진 augementation 하자~~ - 다만, regression인 만큼 가장자리 어떻게 처리할지 생각해보자. - 정사각형 사진이기에 90도 돌린사진, 180도 돌린사진, 270도 돌린사진 이렇게만 해도 데이터가 증식될거같음 -  
**그런데!!!!!** 생각해봐야할것이.....  
위도, 경도 등 여러 요소가 존재할텐데 돌려버리면 원래는 비 안오는거를 비온다고 볼 수도 있을 듯 싶다. 쉽게 예를 들면 I 모양일때는 비가 오고, ㅡ 모양일때는 비가안오는데, 돌리는 바람에 안오는걸 온다고 할 수도 있을 것 같다. - 이건 테스트해볼수밖에 없을 듯 싶다. 
9. pretrained 모델 활용방안 생각하기 - 최소 사진크기가 존재하는데 우리는 40x40이라 작은편.. 이걸 40x40을 이어 붙여서 200x200으로 만들어서 넣으면 될거같은데 시도해보자.
10. 카테고리형 변수 원핫인코딩

| 리더보드(5부터) | mae / fscore | mae | 비고 |
|---|:---:|---:|---:|
| 1.93737 | 1.8744 | 0.2451 |  |
| 1.75177 | 1.7016 | 0.2384 |  |
| 1.61793 | 1.4234 | 0.0832 | 리더보드랑 metric 점수랑 차이 많이 남. 해결해야함. |